In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

2024-10-23 11:08:43.940526: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 11:08:43.940596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 11:08:43.942015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 11:08:43.952483: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 11:08:45.108254: W tensorflow/compiler/tf2

In [2]:
train = ImageDataGenerator(rescale = 1/255)
validation = ImageDataGenerator(rescale = 1/255)

In [3]:
full_train = tf.keras.preprocessing.image_dataset_from_directory("EEdataset1/Dataset/Train")
full_test = tf.keras.preprocessing.image_dataset_from_directory("EEdataset1/Dataset/Test")
full_val = tf.keras.preprocessing.image_dataset_from_directory("EEdataset1/Dataset/Validation")

Found 140002 files belonging to 2 classes.


2024-10-23 11:08:55.555709: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Found 10905 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.


In [4]:
ds_train = full_train
ds_val = full_val
ds_test = full_test

In [5]:
# define the batch size and print the image labels
#import tensorflow_datasets as tfds

batch_size = 64 # Dividing the subsets of the data into smaller chunks that our model can be able to understand.

dataset_name = ds_train
class_names = full_train.class_names
print(class_names) #Display the names of the subfolder that are in the main folder.

['Fake', 'Real']


In [6]:
#resize your images and standardize your data to make it for a neural network
#Standardize your data
size = (90,90)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_val = ds_val.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

In [7]:
NUM_CLASSES = len(class_names)

In [8]:
len(class_names)

2

In [9]:
#One_hot / categorical encoding
def input_preprocess(image, label):
  label = tf.one_hot(label, NUM_CLASSES)
  return image, label

ds_train = ds_train.map(
    input_preprocess, num_parallel_calls = tf.data.AUTOTUNE
)

ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(input_preprocess)
ds_val = ds_val.prefetch(tf.data.AUTOTUNE)

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[90,90,3]), #1
    tf.keras.layers.RandomFlip(mode='horizontal'),         #2
    tf.keras.layers.RandomFlip(mode='vertical'),           #3 
    tf.keras.layers.RandomRotation(factor=0.05),           #4
    tf.keras.layers.RandomZoom(height_factor=0.05),        #5
    tf.keras.layers.RandomContrast(factor = 0.1),          #6
    tf.keras.layers.RandomBrightness(factor = 0.2),        #7 
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), #8
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), #9
    tf.keras.layers.BatchNormalization(),          #10
    tf.keras.layers.MaxPooling2D((2,2)),            #11 
    tf.keras.layers.Dropout(0.2),                    #12 
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), #13
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), #14
    tf.keras.layers.BatchNormalization(), #15
    tf.keras.layers.MaxPooling2D((2,2)), #16
    tf.keras.layers.Dropout(0.3), #17
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), #18
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',  kernel_initializer='he_uniform', padding='same'), #19
    tf.keras.layers.BatchNormalization(), #20
    tf.keras.layers.MaxPooling2D((2,2)), #21
    tf.keras.layers.Dropout(0.4), #21
    tf.keras.layers.Flatten(), #22
    tf.keras.layers.Dense(128, activation = 'relu',  kernel_initializer='he_uniform'), #23
    tf.keras.layers.BatchNormalization(), #24
    tf.keras.layers.Dropout(0.5), #25
    tf.keras.layers.Dense(2, activation = 'softmax')#26
])

In [11]:
#Compile the model
optimizer = optimizers.Adam(epsilon=0.001)
model.compile(
    optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy", tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.Precision(name = 'precision')
                                                                      ]
)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 90, 90, 3)         0         
                                                                 
 random_flip_1 (RandomFlip)  (None, 90, 90, 3)         0         
                                                                 
 random_rotation (RandomRot  (None, 90, 90, 3)         0         
 ation)                                                          
                                                                 
 random_zoom (RandomZoom)    (None, 90, 90, 3)         0         
                                                                 
 random_contrast (RandomCon  (None, 90, 90, 3)         0         
 trast)                                                          
                                                                 
 random_brightness (RandomB  (None, 90, 90, 3)         0

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
es = EarlyStopping(monitor = "val_accuracy", min_delta = 0.01, patience = 10, verbose = 1)
class CustomModelCheckpoint(Callback):
    def __init__(self, filepath, save_freq):
        super(CustomModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.save_freq = save_freq
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:  # Save on specific iterations (1-indexed)
            self.model.save(self.filepath.format(epoch=epoch + 1))

model_cp = ModelCheckpoint(filepath = 'Keras/90x90Best.keras', monitor = "val_accuracy",
                           save_best_only = True,
                           save_weights_only = False,
                           verbose = 1)
# Define your custom checkpoint for specific iterations
specific_iteration_cp = CustomModelCheckpoint(filepath='Keras/90x90Epoch.keras',
                                              save_freq=1)  
import json
import os
import tensorflow as tf  # Ensure you have TensorFlow imported

# Define the custom callback
class HistorySaver(tf.keras.callbacks.Callback):
    def __init__(self, file_path):
        super().__init__()
        self.file_path = file_path
        self.history = []  # Initialize as a list
        self.last_epoch = 0  # Initialize last_epoch
        
        # Load existing history if the file exists
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)  # Load last completed epoch
                
    def on_epoch_end(self, epoch, logs=None):
        # Append the current epoch's logs to the history
        self.history.append({**logs, 'epoch': epoch + 1})  # Store epoch as 1-indexed

        # Save the updated history to the JSON file
        with open(self.file_path, 'w') as file:
             json.dump({'history': self.history, 'last_epoch': epoch + 1}, file, indent=4)

# Usage
file_path = '90x90history.json'
history_saver = HistorySaver(file_path)
# Load the last completed epoch to start from there
start_epoch = history_saver.last_epoch



In [ ]:
#train the model
epochs = 100
hist = model.fit(ds_train, epochs = epochs, validation_data = ds_val, callbacks = [model_cp, specific_iteration_cp, history_saver], batch_size = 64, verbose = 1)

Epoch 1/100
4375/4376 [============================>.] - ETA: 0s - loss: 0.4051 - accuracy: 0.8144 - recall: 0.8144 - precision: 0.8144
Epoch 1: val_accuracy improved from -inf to 0.83410, saving model to Keras/90x90Best.keras
4376/4376 [==============================] - 997s 227ms/step - loss: 0.4051 - accuracy: 0.8144 - recall: 0.8144 - precision: 0.8144 - val_loss: 0.4261 - val_accuracy: 0.8341 - val_recall: 0.8341 - val_precision: 0.8341
Epoch 2/100
4375/4376 [============================>.] - ETA: 0s - loss: 0.2357 - accuracy: 0.9024 - recall: 0.9024 - precision: 0.9024
Epoch 2: val_accuracy improved from 0.83410 to 0.90433, saving model to Keras/90x90Best.keras
4376/4376 [==============================] - 999s 228ms/step - loss: 0.2357 - accuracy: 0.9024 - recall: 0.9024 - precision: 0.9024 - val_loss: 0.2369 - val_accuracy: 0.9043 - val_recall: 0.9043 - val_precision: 0.9043
Epoch 3/100
4375/4376 [============================>.] - ETA: 0s - loss: 0.1879 - accuracy: 0.9249 - reca

In [ ]:
test = ds_test
loss, accuracy, precision, recall = model.evaluate(test)
print('Test accuracy :', accuracy)
print('recall :', recall)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have your history_saver instance
# Extract accuracy and validation accuracy from the history saved in history_saver
train_accuracies = [entry['accuracy'] for entry in history_saver.history]
val_accuracies = [entry['val_accuracy'] for entry in history_saver.history]

# Create a range of epochs based on the length of the accuracies
epochs_range = range(1, len(train_accuracies) + 1)

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_accuracies, label='Training Accuracy', marker='o')
plt.plot(epochs_range, val_accuracies, label='Validation Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.xticks(epochs_range)  # Optional: Set x-ticks to be every epoch
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have your history_saver instance
# Extract accuracy and validation accuracy from the history saved in history_saver
train_loss = [entry['loss'] for entry in history_saver.history]
val_loss = [entry['val_loss'] for entry in history_saver.history]

# Create a range of epochs based on the length of the accuracies
epochs_range = range(1, len(train_loss) + 1)

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_loss, label='Training Loss', marker='o')
plt.plot(epochs_range, val_loss, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(epochs_range)  # Optional: Set x-ticks to be every epoch
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have your history_saver instance
# Extract accuracy and validation accuracy from the history saved in history_saver
train_precision = [entry['precision'] for entry in history_saver.history]
val_precision = [entry['val_precision'] for entry in history_saver.history]

# Create a range of epochs based on the length of the accuracies
epochs_range = range(1, len(train_precision) + 1)

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_precision, label='Training Precison', marker='o')
plt.plot(epochs_range, val_precision, label='Validation Precison', marker='o')
plt.title('Training and Validation Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.xticks(epochs_range)  # Optional: Set x-ticks to be every epoch
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have your history_saver instance
# Extract accuracy and validation accuracy from the history saved in history_saver
train_recall = [entry['recall'] for entry in history_saver.history]
val_recall = [entry['val_recall'] for entry in history_saver.history]

# Create a range of epochs based on the length of the accuracies
epochs_range = range(1, len(train_recall) + 1)

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_recall, label='Training Recall', marker='o')
plt.plot(epochs_range, val_accuracies, label='Validation Recall', marker='o')
plt.title('Training and Validation Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.xticks(epochs_range)  # Optional: Set x-ticks to be every epoch
plt.legend()
plt.grid(True)
plt.show()
